In [185]:
import pandas as pd
import string
import re
from stemming.porter2 import stem
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

corpus1 = pd.read_csv('../1_1_EPIE/idioms.csv')
corpus2 = pd.read_csv('../1_2_dictionary/idioms.csv')
corpus3 = pd.read_csv('../1_3_bookcorpus/idioms.csv')

[nltk_data] Downloading package stopwords to /Users/chany/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [186]:
corpus1 = corpus1.rename(columns={
    'idiom': 'idiom_EPIE',
    'meaning': 'meaning_EPIE',
    'example': 'example_EPIE',
    'tag': 'tag_EPIE'
})

corpus2 = corpus2.rename(columns={
    'idiom': 'idiom_dictionary',
    'meaning': 'meaning_dictionary',
    'example': 'example_dictionary',
})

corpus3 = corpus3.rename(columns={
    'idiom': 'idiom_bookcorpus',
    'definition': 'meaning_bookcorpus'
})
corpora = [corpus1, corpus2, corpus3]

In [187]:
pronouns = {
    'I', 'you', 'You', 'he', 'He', 'she', 'She', 'it', 'It', 'we', 'We', 'they', 'They', # subjective
    'me', 'Me', 'him', 'Him', 'her', 'Her', 'us', 'Us', 'them', 'Them', # objective
    'my', 'My', 'your', 'Your', 'his', 'His', 'our', 'Our', 'their', 'Their', # possessive, adj
    'mine', 'Mine', 'yours', 'Yours' 'hers', 'Hers', 'ours', 'Ours', 'theirs', 'Theirs', # possessive, n
    'pron', 'own'
}
determiners = {
    'the', 'The', 'a', 'A', 'an', 'An', 'this', 'This', 'that', 'That', 'these', 'These', 'those', 'Those'
}
auxes = {'is', 'was', 'are', 'were', 'be', 'being', 'been'}
props = {'in', 'at', 'on', 'of', 'by', 'for', 'by', 'to'}
extras = {'ll'}
stop_ignore = {'between', 'all', 'same', 'but', 'off', 'once', 'have', 'until', 'below', 'through', 'while', 
        'above', 'both', 'up', 'out', 'under', 'against', 'further', 'few', 'most', 'down'}
stop_words = pronouns | determiners | auxes | props | extras | set(stopwords.words('english')) - stop_ignore
def trim_idiom(idiom):
    idiom = idiom.lower()
    idiom = idiom.replace('one\'s ', '')
    idiom = idiom.replace('-', ' ')
    idiom = idiom.replace('\'', ' ')
    idiom = idiom.replace('/', ' ')
    idiom = idiom.translate(str.maketrans('', '', string.punctuation))
    idiom = idiom.split()
    idiom = set([stem(word) for word in idiom])
    idiom -=  stop_words
    return frozenset(idiom)
    

In [188]:
corpus1['idiom_set'] = corpus1['idiom_EPIE'].apply(trim_idiom)
corpus2['idiom_set'] = corpus2['idiom_dictionary'].apply(trim_idiom)
corpus3['idiom_set'] = corpus3['idiom_bookcorpus'].apply(trim_idiom)

In [189]:
corpus = pd.concat([corpus1, corpus2, corpus3])
corpus = corpus.groupby('idiom_set').agg({
    'idiom_EPIE': 'first',
    'meaning_EPIE': 'first',
    'example_EPIE': 'first',
    'tag_EPIE': 'first',
    'idiom_dictionary': 'first',
    'meaning_dictionary': 'first',
    'example_dictionary': 'first',
    'idiom_bookcorpus': 'first',
    'meaning_bookcorpus': 'first'
}).reset_index()

In [190]:
corpus.to_csv('hey.csv', index=False)

In [191]:
def postprocess(row):
    row['dupl'] = 0
    if row['idiom_EPIE'] and not row['idiom_dictionary'] and not row['idiom_bookcorpus']: 
        row['idiom'] = row['idiom_EPIE']
        row['meaning'] = row['meaning_EPIE']
        row['example'] = row['example_EPIE']
        row['tag'] = row['tag_EPIE']
        row['source'] = 'EPIE'
    elif not row['idiom_EPIE'] and row['idiom_dictionary'] and not row['idiom_bookcorpus']: 
        row['idiom'] = row['idiom_dictionary']
        row['meaning'] = row['meaning_dictionary']
        row['example'] = row['example_dictionary']
        row['source'] = 'dictionary'
    elif not row['idiom_EPIE'] and not row['idiom_dictionary'] and row['idiom_bookcorpus']: 
        row['idiom'] = row['idiom_bookcorpus']
        row['meaning'] = row['meaning_bookcorpus']
        row['source'] = 'bookcorpus'
    else:
        row['idiom'] = row['idiom_dictionary']
        row['meaning'] = row['meaning_dictionary']
        row['example'] = row['example_dictionary']
        row['source'] = 'dictionary'
        row['dupl'] = 1
    return row

In [192]:
# corpus = corpus.apply(postprocess, axis=1)
# corpus[~corpus['dupl']][['idiom_set', 'idiom']].reset_index().to_csv('examine.csv', index=False)

In [193]:
corpus = corpus.apply(postprocess, axis=1)[['idiom', 'meaning', 'example', 'source', 'dupl', 'tag',
    'idiom_set', 
    'idiom_EPIE', 'meaning_EPIE', 'example_EPIE',
    'idiom_dictionary', 'meaning_dictionary', 'example_dictionary',
    'idiom_bookcorpus', 'meaning_bookcorpus',
]]
corpus = corpus[corpus['idiom'].notnull()].reset_index(drop=True)

In [194]:
corpus.to_csv('idioms.csv', index=False)

In [195]:
corpus[corpus['dupl']==1].to_csv('dupl.csv', index=False)

In [196]:
corpus

,idiom,meaning,example,source,dupl,tag,idiom_set,idiom_EPIE,meaning_EPIE,example_EPIE,idiom_dictionary,meaning_dictionary,example_dictionary,idiom_bookcorpus,meaning_bookcorpus
0,by and by,soon; after a while.,"By and by, she paused to make sure everyone wa...",dictionary,1,NaN,(),now and then,occasionally; from time to time.,"She said it was br it was , I did not think it...",by and by,soon; after a while.,"By and by, she paused to make sure everyone wa...",None,None
1,the bottom line,"ing: in the final analysis, the most important...",The bottom line is that pregnant women's healt...,dictionary,1,NaN,"(line, bottom)",[pron] bottom line is,the most important point or conclusion.,"It 's something to do with relating to it , bu...",the bottom line,"ing: in the final analysis, the most important...",The bottom line is that pregnant women's healt...,None,None
2,every nook and cranny,to look for something everywhere,The lawyers searched every nook and cranny to ...,dictionary,0,NaN,"(nook, everi, cranni)",None,None,None,every nook and cranny,to look for something everywhere,The lawyers searched every nook and cranny to ...,None,None
3,in the buff,naked,The model created a sensation when she posed f...,dictionary,0,NaN,(buff),None,None,None,in the buff,naked,The model created a sensation when she posed f...,None,None
4,it's anyone's call,a competition where the outcome is difficult t...,I think this year's election would be anyone's...,dictionary,0,NaN,(anycal),None,None,None,it's anyone's call,a competition where the outcome is difficult t...,I think this year's election would be anyone's...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,turn a blind eye,deliberately overlook,"As a police officer, you cannot turn a blind e...",dictionary,1,NaN,"(turn, blind, eye)",turn [pron] blind eye,ignore something that one should not ignore.,"I 'm , I 'm , quite amazed that , that , the q...",turn a blind eye,deliberately overlook,"As a police officer, you cannot turn a blind e...",None,None
1960,trouble and strife,a term of endearment used to refer to one's sp...,All the trouble and strife is behind you today...,EPIE,0,O O B-IDIOM I-IDIOM I-IDIOM O O O O O O O O O ...,"(troubl, strife)",trouble and strife,a term of endearment used to refer to one's sp...,All the trouble and strife is behind you today...,None,None,None,None,None
1961,trip [pron] light fantastic,dance joyfully and energetically.,"IT PROBABLY never existed , the England where ...",EPIE,0,O O O O O O O O O O O O O O O O O O O O B-IDIO...,"(fantast, light, trip)",trip [pron] light fantastic,dance joyfully and energetically.,"IT PROBABLY never existed , the England where ...",None,None,None,None,None
1962,tour de force,an impressive and remarkable feat or accomplis...,"Contradicting their view , however , Pia Hayes...",EPIE,0,O O O O O O O O O O O O O O O O O O O O O O O ...,"(tour, forc, de)",tour de force,an impressive and remarkable feat or accomplis...,"Contradicting their view , however , Pia Hayes...",None,None,None,None,None
